In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Flatten,  concatenate
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout,AveragePooling2D, GlobalAveragePooling2D
from tensorflow.keras.callbacks import ModelCheckpoint
import matplotlib.pylab as plt
import scipy.io
import numpy as np
import pandas as pd
from skimage.transform import resize
import pywt

In [ ]:
re_per_prti_df = pd.DataFrame()
X_sig_data = []

for nprti in range(1, 41):
    file_name = 'Data_Preprocessed_P%02d.mat'% nprti
    EEG_file = scipy.io.loadmat(file_name)
    EEG_sig_data = EEG_file['joined_data']
    EEG_label = EEG_file['labels_selfassessment']
    per_prti_df = pd.DataFrame()
    
    # short video에 대해서만 eeg 신호 가져오기
    for nvideo in range(16):
        globals()['EEG_{}_video_{}_df'.format(nprti, nvideo+1)] = pd.DataFrame(EEG_sig_data[0][nvideo][:,:14])
        

        if globals()['EEG_{}_video_{}_df'.format(nprti, nvideo+1)].isnull().any().any() == True:
            print('참가자',nprti, '비디오', nvideo+1)
        
        else:
            sig = globals()['EEG_{}_video_{}_df'.format(nprti, nvideo+1)].values.shape[0]
            X_sig_video_data = globals()['EEG_{}_video_{}_df'.format(nprti, nvideo+1)].values
            X_sig_data.append(X_sig_video_data)

            per_video_df = pd.DataFrame([EEG_label[0][nvideo][0]],
                     columns=['arousal', 'valence' ,  'dominance',  'liking', 'familiartiy', 'neutral', 
                              'disgust','happiness','surprise', 'anger','fear', 'sadness'])
            per_prti_df = pd.concat([per_prti_df, per_video_df], ignore_index = True)
    re_per_prti_df = pd.concat([re_per_prti_df, per_prti_df], ignore_index = True)

참가자 12 비디오 5
참가자 21 비디오 2
참가자 21 비디오 11
참가자 22 비디오 16
참가자 23 비디오 1
참가자 23 비디오 5
참가자 23 비디오 7
참가자 23 비디오 9
참가자 23 비디오 12
참가자 24 비디오 1
참가자 24 비디오 8
참가자 24 비디오 12
참가자 24 비디오 13
참가자 24 비디오 14
참가자 24 비디오 15
참가자 24 비디오 16
참가자 33 비디오 1
참가자 33 비디오 2
참가자 33 비디오 3
참가자 33 비디오 7
참가자 33 비디오 8
참가자 33 비디오 9
참가자 33 비디오 10
참가자 33 비디오 11
참가자 33 비디오 13
참가자 33 비디오 16


In [ ]:
re_per_prti_df

,arousal,valence,dominance,liking,familiartiy,neutral,disgust,happiness,surprise,anger,fear,sadness
0,2.938568,5.000000,8.235496,1.000000,1.000000,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,4.877136,6.542664,4.849832,9.000000,9.000000,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,5.000000,3.238904,2.774744,1.327648,1.000000,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,2.965872,8.590440,8.098976,8.208192,9.000000,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,6.460752,1.000000,2.856656,1.000000,9.000000,0.0,1.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
609,6.044072,9.000000,2.925424,9.000000,1.000000,0.0,0.0,0.0,0.0,1.0,0.0,1.0
610,6.342376,7.589832,3.955936,7.806784,7.291528,0.0,0.0,0.0,0.0,1.0,0.0,0.0
611,5.000000,6.667800,5.000000,6.586440,9.000000,0.0,0.0,0.0,0.0,1.0,0.0,0.0
612,5.555936,6.288136,4.308472,7.264408,1.000000,0.0,0.0,0.0,0.0,1.0,0.0,1.0


In [ ]:
emotion_label_df = re_per_prti_df.copy()
emotion_label_df = emotion_label_df.drop(emotion_label_df.columns[2:], axis=1)
emotion_label_df

,arousal,valence
0,2.938568,5.000000
1,4.877136,6.542664
2,5.000000,3.238904
3,2.965872,8.590440
4,6.460752,1.000000
...,...,...
609,6.044072,9.000000
610,6.342376,7.589832
611,5.000000,6.667800
612,5.555936,6.288136


In [ ]:
Aro_bi_emotion_label_df = emotion_label_df[['arousal']].copy()
Aro_bi_emotion_label_df[emotion_label_df['arousal'] <= 5] = 0
Aro_bi_emotion_label_df[emotion_label_df['arousal'] > 5] = 1
Aro_bi_emotion_label_df

,arousal
0,0.0
1,0.0
2,0.0
3,0.0
4,1.0
...,...
609,1.0
610,1.0
611,0.0
612,1.0


In [ ]:
val_bi_emotion_label_df = emotion_label_df[['valence']].copy()
val_bi_emotion_label_df[emotion_label_df['valence'] <= 5] = 0
val_bi_emotion_label_df[emotion_label_df['valence'] > 5] = 1
val_bi_emotion_label_df

,valence
0,0.0
1,1.0
2,0.0
3,1.0
4,0.0
...,...
609,1.0
610,1.0
611,1.0
612,1.0


In [ ]:
y_aro_label = Aro_bi_emotion_label_df['arousal'].values
y_vle_label = val_bi_emotion_label_df['valence'].values

In [ ]:
from sklearn.model_selection import train_test_split

X_aro_train_sig, X_aro_val_sig, y_aro_train_label, y_aro_val_label = train_test_split(X_sig_data, y_aro_label, 
                                                                     test_size=0.2, shuffle=True, stratify = y_aro_label,  random_state=1234) 
X_aro_val_sig, X_aro_test_sig, y_aro_val_label, y_aro_test_label = train_test_split(X_aro_val_sig, y_aro_val_label, 
                                                                   test_size=0.5, shuffle=True, stratify = y_aro_val_label, random_state=1234)


X_vle_train_sig, X_vle_val_sig, y_vle_train_label, y_vle_val_label = train_test_split(X_sig_data, y_vle_label, 
                                                                     test_size=0.2, shuffle=True, stratify = y_vle_label, random_state=1234) 
X_vle_val_sig, X_vle_test_sig, y_vle_val_label, y_vle_test_label = train_test_split(X_vle_val_sig, y_vle_val_label, 
                                                                   test_size=0.5, shuffle=True, stratify = y_vle_val_label, random_state=1234)

In [ ]:
# label 저장
import pickle

with open('/y_aro_train_label.pkl', 'wb') as f:
    pickle.dump(y_aro_train_label, f)
    
with open('/y_aro_val_label.pkl', 'wb') as f:
    pickle.dump(y_aro_val_label, f)
    
with open('/y_aro_test_label.pkl', 'wb') as f:
    pickle.dump(y_aro_test_label, f)


with open('/y_vle_train_label.pkl', 'wb') as f:
    pickle.dump(y_vle_train_label, f)
    
with open('/y_vle_val_label.pkl', 'wb') as f:
    pickle.dump(y_vle_val_label, f)
    
with open('/y_vle_test_label.pkl', 'wb') as f:
    pickle.dump(y_vle_test_label, f)

In [ ]:
# cwt_data
from scipy.signal import butter, lfilter
import os
import librosa
import librosa.display

def butter_bandpass(lowcut, highcut, fs, order=5):
    nyq = 0.5*fs
    low = lowcut/nyq
    high = highcut/nyq
    b, a = butter(order, [low, high], btype='band')
    return b,a 

# 4-45 Hz bandpass
def butter_bandpass_filter(data, lowcut, highcut, fs, order=5):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = lfilter(b, a, data)
    return y

def scalogram(x, n_scale = np.arange(2,27), wavelet='morl', sampling_period = 1/128):
    for prti in range(len(x)):
        for ch in range(14):
            x_bandpass = butter_bandpass_filter(x[prti][:,ch], 4,45,128,16)
            coef, freq = pywt.cwt(x_bandpass, n_scale, wavelet, sampling_period)
            fig, ax = plt.subplots(figsize=(7,5))
            im = ax.imshow(abs(np.log2(abs(coef))),cmap = plt.cm.jet, interpolation='bilinear',  aspect = 'auto',vmin=0, vmax=12)
            plt.xticks([])
            plt.yticks([])
            plt.savefig('/cwt_%d.jpg' % (14*(prti)+(ch+1)))
            plt.close()

scalogram(X_aro_train_sig)
#scalogram(X_aro_val_sig)
#scalogram(X_aro_test_sig)

#scalogram(X_vle_train_sig)
#scalogram(X_vle_val_sig)
#scalogram(X_vle_test_sig)